In [11]:
import os
import gdal
import glob

import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union

import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
def rec_find(path, all_f):
    global count
    list1 = os.listdir(path)
    for l in list1:
        if os.path.isdir(path + l + "/"):
            all_f = rec_find(path + l + "/", all_f)
        else:
            if l.split(".")[-1] == 'shp':
                count += 1
                print("Found {}".format(count) + " paths")
                all_f.append(path + l)

    return all_f

In [13]:
def poly_from_utm(polygon, transform):
    poly_pts = []

    # make a polygon from multipolygon
    poly = cascaded_union(polygon)
    for i in np.array(poly.exterior.coords):

        # transfrom polygon to image crs, using raster meta
        poly_pts.append(~transform * tuple(i))

    # make a shapely Polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

In [14]:
classes = ["Многоквартирное жилое здание",
           "Частный дом",
           "Промышленные зоны",
           "Прочие нежилые здания"]

dict_names = {"Многоквартирное жилое здание": 1,
              "Частный дом": 2,
              "Промышленные зоны": 3,
              "Прочие нежилые здания": 4}

dict_shape = {key: [] for key in classes}

In [15]:
# path = "/mnt/datastore/GIS/ROKS/ZU/ann_OKS/"
# raster_path_data = "/mnt/datastore/GIS/ROKS/ZU/raster/"

path = "/mnt/datastore/GIS/ROKS/Rosreestr-Satelite/Razmetka"
raster_path_data = "/mnt/datastore/GIS/ROKS/Rosreestr-Satelite"

count = 0
shp_paths = []
shp_paths = rec_find(path, shp_paths)
# shp_paths = ["/mnt/datastore/GIS/ROKS/trainGSK60/ann/GSKN-39-3-113-d/"]

In [10]:
ls '/mnt/datastore/GIS/ROKS/Rosreestr-Satelite/Razmetka/RP1_2017-05-03_Krasnodar_1/'

RP1_2017-05-03_Krasnodar_1.cpg*  RP1_2017-05-03_Krasnodar_1.qpj*
RP1_2017-05-03_Krasnodar_1.dbf*  RP1_2017-05-03_Krasnodar_1.shp*
RP1_2017-05-03_Krasnodar_1.prj*  RP1_2017-05-03_Krasnodar_1.shx*


In [16]:
shp_paths

[]

In [27]:
for shp in tqdm(shp_paths[7:]):
    dict_shape = {key: [] for key in classes}
    if shp.split("/")[-2].split("_")[-2] == "Kungur":
        base_shp = "ZU_" + \
            shp.split("/")[-2].split("_")[-2] + "_" + \
            shp.split("/")[-2].split("_")[-1]
    else:
        base_shp = "ZU_" + shp.split("/")[-2].split("_")[-1]
#     if base_shp == "O-40-065-(232-д)_ЦОФП":
#         base_shp = base_shp
#     elif base_shp == "O-40-077-(009-г)_ЦОФП_2017_ГСК2011":
#         base_shp = base_shp[:-18]
#     else:
#         base_shp = base_shp[:-13]
    raster_path = raster_path_data + base_shp + "/BLU.tif"
    print(raster_path)
    out_path = raster_path_data + base_shp + "/"
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    raster_img = reshape_as_image(raster_img)
    try:
        train_df = gpd.read_file(shp)
        im_size = (raster_meta['height'], raster_meta['width'])
        for num, row in train_df.iterrows():
            try:
                column = row["Type"]
            except:
                column = row["type"]
            if column == "Прочие нежелые здания":
                column = "Прочие нежилые здания"
            try:
                if row['geometry'].geom_type == 'Polygon':
                    poly = poly_from_utm(
                        row['geometry'], raster_meta['transform'])
                else:
                    for p in row['geometry']:
                        poly = poly_from_utm(p, raster_meta['transform'])
            except Exception as e:
                print(e)
                continue

            dict_shape[column].append(poly)
    except:
        print(base_shp)
        train_df = gpd.read_file(shp, encoding="Windows-1251")
        im_size = (raster_meta['height'], raster_meta['width'])
        for num, row in train_df.iterrows():
            try:
                column = row["Type"]
            except:
                column = row["type"]
            if column == "Прочие нежелые здания":
                column = "Прочие нежилые здания"
            try:
                if row['geometry'].geom_type == 'Polygon':
                    poly = poly_from_utm(
                        row['geometry'], raster_meta['transform'])
                else:
                    for p in row['geometry']:
                        poly = poly_from_utm(p, raster_meta['transform'])
            except Exception as e:
                print(e)
                continue

            dict_shape[column].append(poly)

    for k, v in dict_shape.items():
        if len(v) > 0:
            mask = rasterize(shapes=v, out_shape=im_size)
#             plt.imshow(mask)
#             plt.show()
        else:
            mask = np.zeros(
                (raster_img.shape[1], raster_img.shape[2]), dtype=np.uint8)

        bin_mask_meta = raster_meta.copy()
        bin_mask_meta.update({'count': 1})
        with rasterio.open(out_path + "10" + str(dict_names[k]) + ".tif", 'w', **bin_mask_meta) as dst:
            dst.write(mask * 255, 1)

  0%|                                                                                                            | 0/6 [00:00<?, ?it/s]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-0-0/BLU.tif


 17%|████████████████▋                                                                                   | 1/6 [00:32<02:41, 32.36s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-0-1/BLU.tif


 33%|█████████████████████████████████▎                                                                  | 2/6 [01:06<02:13, 33.27s/it]

...

 67%|██████████████████████████████████████████████████████████████████▋                                 | 4/6 [02:11<01:08, 34.36s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-1-1/BLU.tif


 83%|███████████████████████████████████████████████████████████████████████████████████▎                | 5/6 [02:22<00:25, 25.99s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-1-2/BLU.tif


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:49<00:00, 28.17s/it]


In [28]:
list_shape = []
for shp in tqdm(shp_paths):
    base_shp = shp.split("/")[-2][3:]
#     if base_shp == "O-40-065-(232-д)_ЦОФП":
#         base_shp = base_shp
#     elif base_shp == "O-40-077-(009-г)_ЦОФП_2017_ГСК2011":
#         base_shp = base_shp[:-18]
#     else:
#         base_shp = base_shp[:-13]
    raster_path = raster_path_data + "ZU" + base_shp + "/BLU.tif"
    print(raster_path)
    out_path = raster_path_data + "ZU" + base_shp + "/"
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        raster_meta = src.meta
    raster_img = reshape_as_image(raster_img)
    try:
        train_df = gpd.read_file(shp)
        im_size = (raster_meta['height'], raster_meta['width'])
        for num, row in train_df.iterrows():
            try:
                if row['geometry'].geom_type == 'Polygon':
                    poly = poly_from_utm(
                        row['geometry'], raster_meta['transform'])
                else:
                    for p in row['geometry']:
                        poly = poly_from_utm(p, raster_meta['transform'])
            except Exception as e:
                print(e)
                continue

            list_shape.append(poly)
    except:
        print(base_shp)
        train_df = gpd.read_file(shp, encoding="Windows-1251")
        im_size = (raster_meta['height'], raster_meta['width'])
        for num, row in train_df.iterrows():
            try:
                if row['geometry'].geom_type == 'Polygon':
                    poly = poly_from_utm(
                        row['geometry'], raster_meta['transform'])
                else:
                    for p in row['geometry']:
                        poly = poly_from_utm(p, raster_meta['transform'])
            except Exception as e:
                print(e)
                continue

            list_shape.append(poly)

    mask = rasterize(shapes=list_shape, out_shape=im_size)

    bin_mask_meta = raster_meta.copy()
    bin_mask_meta.update({'count': 1})
    with rasterio.open(out_path + "100" + ".tif", 'w', **bin_mask_meta) as dst:
        dst.write(mask * 255, 1)

  0%|                                                                                                           | 0/13 [00:00<?, ?it/s]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Glyadenovo-0-0/BLU.tif


  8%|███████▌                                                                                           | 1/13 [00:11<02:23, 11.95s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Glyadenovo-0-1/BLU.tif


 15%|███████████████▏                                                                                   | 2/13 [00:15<01:19,  7.19s/it]

...

 85%|██████████████████████████████████████████████████████████████████████████████████▉               | 11/13 [01:25<00:09,  4.86s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-1-1/BLU.tif


 92%|██████████████████████████████████████████████████████████████████████████████████████████▍       | 12/13 [01:26<00:03,  3.87s/it]

/mnt/datastore/GIS/ROKS/ZU/raster/ZU_Kungur_1-1-2/BLU.tif


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:28<00:00,  6.77s/it]


In [35]:
from sys import getdefaultencoding
getdefaultencoding()

'utf-8'

In [36]:
column.encode("'ascii'", errors='xmlcharrefreplace')

b'&#1055;&#1088;&#1086;&#1095;&#1080;&#1077; &#1085;&#1077;&#1078;&#1080;&#1083;&#1099;&#1077; &#1079;&#1076;&#1072;&#1085;&#1080;&#1103;'